<font size="5">**Algorytmy danych geoprzestrzennych**</font><br>
<font size="4">Przetwarzanie danych wektorowych</font>

<font size="4">Krzysztof Dyba</font>

In [1]:
import os
from qgis.core import QgsVectorLayer

# wczytanie warstwy wektorowej
sciezka_do_pliku = os.path.join("algorytmy-geoprzestrzenne", "dane", "powiaty.gpkg")
wektor = QgsVectorLayer(sciezka_do_pliku, "powiaty", "ogr")
print(wektor.isValid())

True


# Dostęp do obiektów

In [38]:
from itertools import islice

# iteracja po obiektach
for feature in islice(wektor.getFeatures(), 5):
    # dostęp do atrybutów obiektów
    attrs = feature.attributes()
    # dostęp do geometrii obiektów
    geom = feature.geometry()
    
    print(attrs)
    # print(geom)

[1, '3012', 'powiat krotoszyński', 'dc54ce6a-5272-45fc-a49c-824dbf170f2c', '2024-09-05T13:41:42+02:00', 712.7437283077061]
[2, '3002', 'powiat czarnkowsko-trzcianecki', 'b2e00ebb-412b-4ae8-9afc-fe68f65bb1ac', '2021-05-11T10:49:15+02:00', 1806.6288683868954]
[3, '3009', 'powiat kolski', '06a4a975-dc89-4ac3-9d51-cf188532908a', '2012-09-27T08:59:03+02:00', 1009.273272845488]
[4, '3029', 'powiat wolsztyński', '6f988fb3-2b2c-467e-9d07-c512d9215c05', '2024-08-27T13:56:13+02:00', 679.340479768619]
[5, '3026', 'powiat śremski', '2fc58bb6-d095-4f37-8f10-874797cb97fb', '2024-09-03T13:42:56+02:00', 573.5639814826102]


# Obliczanie powierzchni poligonów

W celu obliczenia powierzchni poligonów musimy wykonać pętle po wszystkich obiektach znajdujących się w warstwie i uzyskać dostęp do ich geometrii. Następnie, należy wykorzystać metodę `area()`, która oblicza pole powierzchni w jednostkach układu współrzędnych warstwy (zazwyczaj są to metry kwadratowe). Opcjonalnie, możemy dokonać konwersji jednostek, np. na kilometry kwadratowe czy hektary. W niniejszym przykładzie zakładamy, że wszystkie obiekty w warstwie posiadają geometrie.

In [33]:
for feature in islice(wektor.getFeatures(), 5):
    area = feature.geometry().area()
    area = area / 1000**2 # konwersja na km^2
    print(area)

712.7437283077061
1806.6288683868954
1009.273272845488
679.340479768619
573.5639814826102


# Dodawanie atrybutów

In [7]:
from qgis.core import QgsField
from qgis.PyQt.QtCore import QMetaType

# dodanie nowego atrybutu do tabeli
new_field = QgsField("pole_km2", QMetaType.Type.Double) # typ zmiennoprzecinkowy
wektor.dataProvider().addAttributes([new_field])
wektor.updateFields()

# rozpoczęcie trybu edycji warstwy
wektor.startEditing()

# indeks nowego atrybutu
area_idx = wektor.fields().indexOf("pole_km2")

# obliczenie powierzchni dla każdego obiektu
for feature in wektor.getFeatures():
    area = feature.geometry().area()
    area = area / 1000**2
    # wprowadzenie wartości do atrybutu obiektu
    wektor.changeAttributeValue(feature.id(), area_idx, area)

# zapisanie zmian
wektor.commitChanges()

True

In [34]:
# weryfikacja
print("pole_km2" in wektor.fields().names())

for feature in islice(wektor.getFeatures(), 5):
    print(feature.attribute("pole_km2"))

True
712.7437283077061
1806.6288683868954
1009.273272845488
679.340479768619
573.5639814826102


# Wybieranie obiektów używając:
## Atrybutów

W zasadzie są dwa sposoby: `selectByExpression`, `QgsFeatureRequest`.

## Lokalizacji

In [36]:
# zapytanie
expression = '"pole_km2" > 1000'

# Use QgsFeatureRequest with the expression
wektor.selectByExpression(expression)

selected_features = wektor.selectedFeatures()
print(len(selected_features)) # liczba obiektów

for feature in selected_features:
    pass
    # print(feature.id()) # ID obiektów

# wybierz obiekty na mapie (działa tylko w QGIS)
# warstwa.selectByIds([feature.id() for feature in selected_features])

12


In [31]:
from qgis.core import QgsExpression, QgsFeatureRequest

expression = QgsExpression('"pole_km2" > 1000')

request = QgsFeatureRequest(expression)
selected_features = wektor.getFeatures(request)
for feature in selected_features:
    pass
    # print(feature.attributes())

In [30]:
from qgis.core import QgsFeatureRequest

expression = '"pole_km2" > 1000'

# stworzenie żądania używając zapytania
request = QgsFeatureRequest().setFilterExpression(expression)

for feature in wektor.getFeatures(request):
    pass
    # print(feature.attributes())

ZADANIA:
- oblicz długość granic i dodaj jako nowy atrybut do warstwy
- oblicz statystki opisowe powierzchni oraz długości
- stwórz nową warstwę z obliczonymi centroidami dla powiatów

TODO:
- Selecting Features by Attribute and Localization
- Buffering Features (Buffer Creation)
- Creating a New Vector Layer (programmatically)
- Reprojecting Layer
- Use QGIS vector tools